<a href="https://colab.research.google.com/github/grandhigowthamramsai123/mrm/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files
uploaded=files.upload()

Saving titanic.csv to titanic (4).csv


In [55]:
df=pd.read_csv('titanic.csv')

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [57]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [58]:
df['Age'].fillna(df['Age'].median(),inplace = True)

/tmp/ipython-input-1750550615.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(),inplace = True)


because the boarded station doesn't create any sense as well as the cabin values are 75 percent missing so tha and even passenger id doesn't make any  sense so that i am droping them name and ticket also doesn't add much to the output

In [59]:
df = df.drop(['PassengerId','Cabin','Embarked','Name','Ticket'], axis=1, errors='ignore')

In [60]:
print(df.groupby('Sex')['Survived'].mean())


Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64


There is the pclass and fare both are interrelated but i will be keeping both
because better optimization because vif is in the accepatable range

based on the survival rate females are given a better survival rate

In [61]:
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1}).fillna(0).astype(int)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [63]:
Train,Valid,Test=np.split(df.sample(frac=1),[int(0.8*len(df)),int(0.9*len(df))])

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [64]:
def mean(Train, x):
    Train[x] = (Train[x] - Train[x].mean()) / Train[x].std()
    return Train[x]

In [65]:
Train['Fare']=mean(Train,'Fare')
Train['Age']=mean(Train,'Age')


Test['Fare']=mean(Test,'Fare')
Test['Age']=mean(Test,'Age')


In [66]:
df.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,0
SibSp,0
Parch,0
Fare,0


In [67]:
def seperate(df):
  x=df.iloc[:, 1:]
  y=df.iloc[:,0]

  return x,y
train_x,train_y=seperate(Train)
valid_x,valid_y=seperate(Valid)
test_x,test_y=seperate(Test)

In [68]:
train_y.value_counts()

,count
Survived,
0,441
1,271


**i am not balancing the class as they don't differ much**



In [69]:
class LogisticRegression:
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.weights = None

    def sigmoid(self, z):
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

    def train(self, epochs=1000, lr=0.01):
        X = self.X.to_numpy()
        Y = np.expand_dims(self.Y.to_numpy(), axis=1)


        X = np.hstack((np.ones((X.shape[0],1)), X))

        self.weights = np.zeros((X.shape[1], 1))

        for epoch in range(epochs):
            z = np.dot(X, self.weights)
            preds = self.sigmoid(z)
            error = preds - Y
            self.weights -= lr * np.dot(X.T, error) / X.shape[0]

    def predict(self, X):
        X = np.hstack((np.ones((X.shape[0],1)), X.to_numpy()))
        z = np.dot(X, self.weights)
        return (self.sigmoid(z) >= 0.5).astype(int)

    def test(self, X, Y):
        preds = self.predict(X)
        Y = np.expand_dims(Y.to_numpy(), axis=1)
        accuracy = (preds == Y).mean()
        print("Accuracy:", accuracy)
        return preds



In [70]:
model = LogisticRegression(train_x, train_y)
model.train(epochs=1000, lr=0.15)
preds = model.test(test_x, test_y)

Accuracy: 0.8444444444444444
